In [1]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
swan_spark_conf.getAll()

[('spark.driver.extraClassPath',
  '/usr/local/lib/python3.9/dist-packages/sparkmonitor/listener_2.12.jar'),
 ('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener')]

In [3]:
config = {
    # Add the first 2 variables from swan_spark_conf
    "spark.extraListeners": "sparkmonitor.listener.JupyterSparkMonitorListener",
    "spark.driver.extraClassPath": "/usr/local/lib/python3.9/dist-packages/sparkmonitor/listener_2.12.jar",
    "spark.kubernetes.namespace": "minio",
    "spark.kubernetes.container.image": "vensav/spark-notebook:3.3.0-scala_2.12-jre_17-slim-buster",
    "spark.executor.instances": "2",
    "spark.executor.memory": "1g",
    "spark.executor.cores": "1",
    "spark.driver.blockManager.port": "7777",
    "spark.driver.port": "2222",
    "spark.driver.host": "jupyter.minio",
    "spark.driver.bindAddress": "0.0.0.0",
}


def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local")
    for key, value in config.items():
        conf.set(key, value)    
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()


In [4]:
def load_config(spark_context: SparkContext):

    S3_HOST_URL = "minio-1662250093-svc.minio:9000"
    S3_ACCESS_KEY = "vensav-developer"
    S3_SECRET_KEY = "vensav123"

    spark_context._jsc.hadoopConfiguration().set('spark.executor.extraJavaOptions=-Dcom.amazonaws.services.s3.enableV4', 'true')
    spark_context._jsc.hadoopConfiguration().set('spark.driver.extraJavaOptions=-Dcom.amazonaws.services.s3.enableV4', 'true')
    spark_context._jsc.hadoopConfiguration().set('spark.hadoop.com.amazonaws.services.s3.enableV4', 'true')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.multipart.size', '104857600')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
    #spark_context._jsc.hadoopConfiguration().set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.endpoint', S3_HOST_URL)
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.access.key', S3_ACCESS_KEY)
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.secret.key', S3_SECRET_KEY)
    spark_context._jsc.hadoopConfiguration().set('fs.s3a.connection.ssl.enabled', 'false')

In [5]:
spark = get_spark_session("test-app", SparkConf())

22/09/09 03:51:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
INFO:SparkMonitorKernel:Client Connected ('127.0.0.1', 57230)


In [6]:
load_config(spark.sparkContext)

In [7]:
try:
    dataFrame = spark.read.json('s3a://test-bucket/*')
    average = dataFrame.groupBy("id").agg({'amount': 'avg'})
    average.show()
except Exception as exp:
    print(exp)

22/09/09 03:51:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---+------------------+
| id|       avg(amount)|
+---+------------------+
|  1|1566.6666666666667|
|  3|            3900.0|
|  2|2733.3333333333335|
+---+------------------+



In [ ]:
spark.stop()